# Chapter 06 실습 2: 한국어 텍스트 분류

## 목표
네이버 영화 리뷰(NSMC) 데이터로 한국어 감성 분석을 구현한다.

## 학습 내용
- 한국어 감성 분석의 특수성(교착어, 형태소)을 이해한다.
- KoNLPy 미설치 환경에서도 실행 가능한 폴백(fallback) 로직을 구현한다.
- 공백 분리와 형태소 분리의 성능 차이를 비교한다.

In [ ]:
# 라이브러리 임포트
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import re

# 한글 폰트 설정 (macOS)
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

print(f"TensorFlow 버전: {tf.__version__}")
tf.random.set_seed(42)
np.random.seed(42)

## NSMC(Naver Sentiment Movie Corpus) 데이터셋

### 데이터셋 소개
- 네이버 영화 리뷰 200,000개 (훈련 150,000 / 테스트 50,000)
- 레이블: 0 (부정), 1 (긍정)
- 출처: [https://github.com/e9t/nsmc](https://github.com/e9t/nsmc)

### 데이터 로드 방법 (실제 파일이 있는 경우)
```python
import pandas as pd

# GitHub에서 다운로드 후 로컬 경로 지정
train_df = pd.read_csv('ratings_train.txt', sep='\t')
test_df  = pd.read_csv('ratings_test.txt',  sep='\t')

# 결측값 제거
train_df = train_df.dropna()
test_df  = test_df.dropna()

train_texts  = train_df['document'].tolist()
train_labels = train_df['label'].tolist()
test_texts   = test_df['document'].tolist()
test_labels  = test_df['label'].tolist()
```

이 실습에서는 실제 NSMC 없이도 실행 가능하도록 샘플 데이터를 하드코딩한다.

In [ ]:
# 샘플 한국어 영화 리뷰 데이터 (NSMC 없이 실행 가능)
# 실제 NSMC와 유사한 구조로 설계된 샘플 데이터

SAMPLE_REVIEWS = [
    # 긍정 리뷰 (label=1)
    ("정말 재미있었어요 배우들의 연기가 너무 좋았습니다",           1),
    ("감동적인 영화 눈물이 절로 났어요",                          1),
    ("스토리가 탄탄하고 연출이 훌륭합니다",                        1),
    ("올해 본 영화 중 최고 강력 추천합니다",                       1),
    ("배우들의 연기력이 정말 뛰어나네요",                          1),
    ("긴장감 넘치는 전개 끝까지 눈을 뗄 수 없었어요",               1),
    ("가족과 함께 보기 좋은 따뜻한 영화",                         1),
    ("명작입니다 두 번 세 번 봐도 좋을 것 같아요",                  1),
    ("OST도 너무 좋고 영상미도 훌륭했습니다",                      1),
    ("반전이 있어서 더욱 재미있었습니다 추천",                      1),
    ("배우들이 모두 열연을 펼쳐서 몰입이 잘 됐어요",                1),
    ("예상보다 훨씬 좋았습니다 감독의 연출력이 대단해요",            1),
    ("이런 영화를 기다렸습니다 정말 완성도가 높아요",               1),
    ("웃음과 감동을 동시에 주는 영화 강추합니다",                   1),
    ("특수효과도 훌륭하고 스토리도 탄탄하네요",                     1),
    ("주인공의 캐릭터가 너무 매력적이었어요",                       1),
    ("시나리오가 정말 잘 짜여져 있어요 감동입니다",                  1),
    ("한국 영화의 저력을 보여주는 걸작",                           1),
    ("처음부터 끝까지 흥미진진하게 봤습니다",                       1),
    ("연출 연기 스토리 모든 면에서 완벽했어요",                     1),
    # 부정 리뷰 (label=0)
    ("완전 실망이에요 돈 아까워",                                  0),
    ("스토리가 엉망이에요 시간 낭비했습니다",                       0),
    ("배우 연기가 너무 어색하고 대사도 유치하네요",                  0),
    ("지루해서 중간에 나올 뻔 했어요",                             0),
    ("억지 전개가 너무 심해요 공감이 안 됩니다",                    0),
    ("기대했는데 너무 실망이에요 비추천",                           0),
    ("이런 영화를 만들어도 되는 건지 최악이에요",                    0),
    ("복잡하기만 하고 아무 감동도 없어요",                          0),
    ("주인공이 너무 답답해서 화가 났어요",                          0),
    ("돈 버리는 영화 절대 비추",                                  0),
    ("개연성이 전혀 없고 결말도 황당해요",                          0),
    ("연출이 조잡하고 특수효과도 형편없어요",                        0),
    ("시간이 아까웠습니다 정말 별로였어요",                         0),
    ("억지 감동 코드가 너무 노골적이에요",                          0),
    ("스토리 구멍이 너무 많아요 논리가 없어요",                     0),
    ("처음에는 기대했는데 점점 실망스러웠어요",                      0),
    ("2시간이 넘는데 볼 게 하나도 없어요",                         0),
    ("캐릭터들이 너무 평면적이고 매력이 없어요",                     0),
    ("감독이 하고 싶은 말이 뭔지 모르겠어요",                       0),
    ("뻔한 내용에 재미도 감동도 없는 영화",                         0),
]

# 데이터를 훈련/검증/테스트로 분할
import random
random.seed(42)

# 데이터 확장 (샘플을 반복하여 충분한 양 확보)
expanded_data = []
for _ in range(50):  # 50회 반복으로 2000개 생성
    shuffled = SAMPLE_REVIEWS.copy()
    random.shuffle(shuffled)
    expanded_data.extend(shuffled)

random.shuffle(expanded_data)

texts  = [item[0] for item in expanded_data]
labels = [item[1] for item in expanded_data]

# 훈련(70%) / 검증(15%) / 테스트(15%) 분할
total    = len(texts)
train_end = int(total * 0.7)
val_end   = int(total * 0.85)

train_texts  = texts[:train_end]
train_labels = np.array(labels[:train_end])
val_texts    = texts[train_end:val_end]
val_labels   = np.array(labels[train_end:val_end])
test_texts   = texts[val_end:]
test_labels  = np.array(labels[val_end:])

print(f"전체 데이터: {total:,d}개")
print(f"  훈련: {len(train_texts):,d}개  |  검증: {len(val_texts):,d}개  |  테스트: {len(test_texts):,d}개")
print(f"  긍정 비율 (훈련): {train_labels.mean():.1%}")
print()
print("샘플 데이터:")
for text, label in zip(texts[:4], labels[:4]):
    print(f"  [{'긍정' if label == 1 else '부정'}] {text}")

In [ ]:
# KoNLPy 형태소 분석 또는 공백 기준 분리 폴백

# 한국어 특수문자 제거 함수
def clean_korean(text):
    """한국어 텍스트 정제: 숫자, 영문, 한글, 공백만 유지"""
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# KoNLPy 설치 여부 확인
USE_KONLPY = False
okt = None

try:
    from konlpy.tag import Okt
    okt = Okt()
    USE_KONLPY = True
    print("KoNLPy(Okt) 로드 성공 → 형태소 분석 사용")
except ImportError:
    print("KoNLPy 미설치 → 공백 기준 분리 사용 (폴백)")
    print("설치 방법: pip install konlpy")

def tokenize(text, use_konlpy=USE_KONLPY):
    """
    텍스트를 토큰으로 분리
    KoNLPy 설치 시 형태소 분석, 미설치 시 공백 분리
    
    Args:
        text: 입력 텍스트
        use_konlpy: KoNLPy 사용 여부
    
    Returns:
        토큰 리스트
    """
    cleaned = clean_korean(text)
    if use_konlpy and okt is not None:
        # Okt 형태소 분석기 사용
        # norm=True: 어휘 정규화 (예: 함ㅎㅎ → 함)
        # stem=True: 어간 추출 (예: 가는 → 가다)
        return okt.morphs(cleaned, norm=True, stem=True)
    else:
        # 공백 기준 단순 분리
        return cleaned.split()

# 토큰화 예시
example_texts = [
    "정말 재미있었어요 배우들의 연기가 너무 좋았습니다",
    "완전 실망이에요 돈 아까워",
]
print("\n=== 토큰화 결과 ===")
for text in example_texts:
    tokens = tokenize(text)
    method = "형태소 분석" if USE_KONLPY else "공백 분리"
    print(f"  원문: '{text}'")
    print(f"  토큰 ({method}): {tokens}")
    print()

# 전체 데이터 토큰화
print("전체 데이터 토큰화 중...")
train_tokenized = [' '.join(tokenize(t)) for t in train_texts]
val_tokenized   = [' '.join(tokenize(t)) for t in val_texts]
test_tokenized  = [' '.join(tokenize(t)) for t in test_texts]
print(f"  토큰화 완료: 훈련 {len(train_tokenized)}개")

In [ ]:
# TextVectorization + Embedding + BiLSTM 모델 구성

# 하이퍼파라미터
MAX_TOKENS  = 5000   # 한국어는 어휘 크기가 작을 수 있어 5000으로 설정
MAX_SEQ_LEN = 50     # 한국어 리뷰는 비교적 짧음
EMBED_DIM   = 64     # 임베딩 차원
LSTM_UNITS  = 32     # LSTM 은닉 유닛
BATCH_SIZE  = 32     # 배치 크기
EPOCHS      = 15     # 학습 에포크

# TextVectorization 레이어 생성 및 어휘 사전 구축
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=MAX_TOKENS,
    output_sequence_length=MAX_SEQ_LEN,
    output_mode='int',
    name='text_vectorizer'
)

# 훈련 데이터로 어휘 사전 구축
vectorizer.adapt(train_tokenized)
vocab = vectorizer.get_vocabulary()
print(f"어휘 사전 크기: {len(vocab)}")
print(f"주요 단어 (상위 20개): {vocab[2:22]}")

# 텍스트 → 정수 시퀀스 변환
x_train = vectorizer(train_tokenized)
x_val   = vectorizer(val_tokenized)
x_test  = vectorizer(test_tokenized)
y_train = train_labels.astype('float32')
y_val   = val_labels.astype('float32')
y_test  = test_labels.astype('float32')

print(f"\n훈련 데이터 형태: {x_train.shape}")

# BiLSTM 모델 구성
model = tf.keras.Sequential([
    # 임베딩 레이어
    tf.keras.layers.Embedding(
        input_dim=len(vocab) + 1,
        output_dim=EMBED_DIM,
        mask_zero=True,
        name='embedding'
    ),
    
    # 양방향 LSTM: 한국어의 문장 앞뒤 문맥 모두 활용
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(LSTM_UNITS, dropout=0.3),
        name='bilstm'
    ),
    
    # 완전 연결층
    tf.keras.layers.Dense(32, activation='relu', name='dense'),
    tf.keras.layers.Dropout(0.4),
    
    # 출력층: 이진 분류
    tf.keras.layers.Dense(1, activation='sigmoid', name='output')
], name='Korean_BiLSTM_Sentiment')

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# 모델 학습 및 평가

# 콜백 설정
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    ),
]

print("한국어 BiLSTM 모델 학습 시작...")
history = model.fit(
    x_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=1
)

# 테스트 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\n테스트 정확도: {test_acc:.4f}")
print(f"테스트 손실:   {test_loss:.4f}")

# 학습 곡선 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
epochs_range = range(1, len(history.history['accuracy']) + 1)

axes[0].plot(epochs_range, history.history['accuracy'],
             'b-o', label='훈련 정확도', linewidth=2)
axes[0].plot(epochs_range, history.history['val_accuracy'],
             'r-s', label='검증 정확도', linewidth=2)
axes[0].axhline(y=test_acc, color='green', linestyle='--',
                label=f'테스트: {test_acc:.4f}')
axes[0].set_xlabel('에포크')
axes[0].set_ylabel('정확도')
axes[0].set_title('정확도 학습 곡선')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(epochs_range, history.history['loss'],
             'b-o', label='훈련 손실', linewidth=2)
axes[1].plot(epochs_range, history.history['val_loss'],
             'r-s', label='검증 손실', linewidth=2)
axes[1].set_xlabel('에포크')
axes[1].set_ylabel('손실')
axes[1].set_title('손실 학습 곡선')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.suptitle(f'한국어 BiLSTM 감성 분석 학습 곡선\n'
             f'(토큰화: {"형태소 분석(Okt)" if USE_KONLPY else "공백 분리"})',
             fontsize=13)
plt.tight_layout()
plt.show()

# 새 리뷰 예측 테스트
def predict_korean_sentiment(text):
    """한국어 리뷰 문자열 → 긍정/부정 예측"""
    tokenized = ' '.join(tokenize(text))
    vec = vectorizer([tokenized])
    prob = model.predict(vec, verbose=0)[0][0]
    label = "긍정" if prob >= 0.5 else "부정"
    return label, float(prob)

print("\n=== 새 리뷰 예측 ===")
new_reviews = [
    "진짜 너무 재미있어요 두 번 보고 싶어요",
    "별로였어요 시간 낭비",
    "배우들이 연기를 정말 잘 하네요",
    "이해가 안 되는 스토리였어요",
]
for review in new_reviews:
    label, prob = predict_korean_sentiment(review)
    print(f"  [{label}({prob:.4f})] {review}")

## 도전 과제

### 1. 형태소 분리 vs 공백 분리 성능 비교

KoNLPy가 설치된 환경에서 두 가지 방법을 비교해 보세요.

```python
# 공백 분리 모델 학습
train_ws  = [' '.join(t.split()) for t in train_texts]  # 공백 분리
# 형태소 분리 모델 학습  
train_ma  = [' '.join(okt.morphs(t, norm=True, stem=True)) for t in train_texts]  # 형태소

# 두 모델의 테스트 정확도 비교
```

예상 결과:
| 토큰화 방법 | 예상 정확도 | 장단점 |
|-------------|-------------|--------|
| 공백 분리 | ~75~80% | 빠르지만 같은 단어의 다른 형태 미인식 |
| 형태소 분석 | ~80~85% | 느리지만 어간 통합으로 성능 향상 |

### 2. 어휘 사전 크기 실험
- `MAX_TOKENS`를 1000, 3000, 5000, 10000으로 변경하며 성능 변화를 분석해 보세요.

### 3. 모델 구조 실험
- 단방향 LSTM vs 양방향 LSTM의 성능 차이를 비교해 보세요.
- LSTM 대신 GRU를 사용해 보세요.

### 4. 실제 NSMC 데이터 적용
아래 명령어로 실제 데이터를 다운로드하여 실행해 보세요:
```bash
wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
```